In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import torchvision
from torch.utils.data import Dataset, SubsetRandomSampler, DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit


import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import seaborn as sns
from tqdm import tqdm

import torch.optim as optim
import torch
import torch.nn as nn


In [2]:
batch_size = 64
classes = list(range(10))


In [3]:
# Download the training dataset
trainset = MNIST(root='data/', train=True,
                      transform=ToTensor(), download=True)

# Download the test dataset
testset = MNIST(root='data/', train=False,
                     transform=ToTensor(), download=True)


In [4]:
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset,
                                                 [train_size, val_size])


In [5]:
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    valset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False)


In [6]:
len(trainset), len(valset), len(testset)

(48000, 12000, 10000)

In [19]:
labels = trainset.dataset.targets[trainset.indices].numpy()
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
_idx, subset_idx = next(split.split(trainset.indices, labels))
new_indices = list(np.array(trainset.indices)[subset_idx])
subset_sampler = SubsetRandomSampler(new_indices)

subset_loader = DataLoader(trainset.dataset, batch_size=64, sampler=subset_sampler)



In [16]:
len(labels), len(new_indices), len(subset_loader), len(trainset.indices)

(48000, 9600, 150, 48000)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(
            in_channels=10, out_channels=20, kernel_size=2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        
        self.fc1 = nn.Linear(in_features=20*5*5, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=10)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = x.flatten(start_dim=1)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        x = self.softmax(x)
        return x


In [20]:
transforms_arr = [
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomRotation(10),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
]
transforms_names = [
    'Resize',
    'LeftRightFlip',
    'Rotation',
    'GaussianNoise',
]
transforms_combined = torchvision.transforms.Compose(
    transforms_arr+[transforms.ToTensor()])

transforms_arr.append(transforms_combined)
transforms_names.append('Combined')
    
    

In [21]:
for transform,transform_name in zip(transforms_arr, transforms_names):
    print('applying',transform_name)
    augmented_subset = []
    for image, label in subset_loader:
        augmented_subset.append(transform(image))
        model = CNN()

applying Resize
applying LeftRightFlip
applying Rotation


KeyboardInterrupt: 